이 과제는 복잡한 컨볼루션 네트워크를 만드는 과제입니다.

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

In [3]:
# Configs
configs = {
    'batch_size': 64,
    'learning_rate': 1e-3,
    'epochs': 20
}

In [2]:
# Dataset
vision_datasets = {split: datasets.CIFAR10(root='data',
                                           train=(split == 'train'),
                                           transform=transforms.ToTensor(),
                                           download=True) for split in ['train', 'val']}

dataset_sizes = {split: len(vision_datasets[split]) for split in ['train', 'val']}

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [6]:
# DataLoader
vision_dataloaders = {split: DataLoader(dataset=vision_datasets[split],
                                        batch_size=configs['batch_size'],
                                        shuffle=(split == 'train')) for split in ['train', 'val']}

for inputs, labels in vision_dataloaders["val"]:
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [13]:
# Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, reduce_image_size=False):
        super().__init__()

        self.residual_block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=out_channels,
                      kernel_size=3,
                      stride=(2 if reduce_image_size else 1),
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels,
                      out_channels=out_channels,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU()
        )
    
    def forward(self, inputs):
        return self.residual_block(inputs)

In [ ]:
# Network
class ResNet18(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3,
                      out_channels=64,
                      kernel_size=7,
                      stride=2,
                      padding=3),
            nn.ReLU()
        )

        self.conv2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, 
                         stride=2, 
                         padding=1),
            ResidualBlock(in_channels=64,
                          out_channels=64),
            ResidualBlock(in_channels=64,
                          out_channels=64)
        )

        self.conv3 = nn.Sequential(
            ResidualBlock(in_channels=64,
                          out_channels=128,
                          reduce_image_size=True),
            ResidualBlock(in_channels=128,
                          out_channels=128)
        )

        self.conv4 = nn.Sequential(
            ResidualBlock(in_channels=128,
                          out_channels=256,
                          reduce_image_size=True),
            ResidualBlock(in_channels=256,
                          out_channels=256)
        )

        self.conv5 = nn.Sequential(
            ResidualBlock(in_channels=256,
                          out_channels=512,
                          reduce_image_size=True),
            ResidualBlock(in_channels=512,
                          out_channels=512)
        )

        self.classifier = nn.Sequential(
            nn.MaxPool2d(kernel_size=7, stride=1),
            nn.Linear(in_features=512,
                      out_features=100)
        )
        
        
    
    def forward(self, inputs):
        pass

In [ ]:
# Model
resnet18 = ResNet18()
print(resnet18)